In [2]:
# coding: utf-8
import pandas as pd
import numpy as np
import data_load as dl

In [27]:
dl.get_data("""
with
random_rate as 
(
    select 
    user_id, avg(result) as random_value, count(distinct insert_time) as attempts
    from tl.game_results
    group by user_id
), 
left_tbl as 
(
    select battle1 as item, user_id, "user"
        , case 
            when result = 0 then 1
            else 0
        end as is_selected
            --, row_number() as id
    from tl.game_results
),
right_tbl as 
(
        select battle2 as item, user_id, "user"
                    , case 
                        when result = 1 then 1
                        else 0
                    end as is_selected
    from tl.game_results
),
pre_items as 
(
    select * from left_tbl

    union all

    select * from right_tbl
),
items as 
(
    select *, row_number() over() as id
    from pre_items
),
global_stat as 
(
    select item
                , sum(is_selected) as selected
                , count(is_selected) as total
                , sum(is_selected)::float / count(is_selected) as select_rate
    from items
    group by item
),
personal_stat as 
(
    select item, user_id
                , sum(is_selected) as selected
                , count(is_selected) as total
                , sum(is_selected)::float / count(is_selected) as select_rate
    from items
    group by item, user_id
),
indiv_stat as 
(
    select ps.*
            , gs.select_rate as global_select_rate
            , case 
                when (ps.select_rate <= 0.05) or (ps.select_rate >= 0.95) then 0
                else  round((100 * abs(gs.select_rate - ps.select_rate))::decimal, 0)
              end as original_score
    from personal_stat ps
    left outer join global_stat gs on 
        (ps.item=gs.item)
)


select user_id, sum(original_score) as original_score, sum(total) as attemps
from  indiv_stat
--where "user"='rosolimo' and item='Айвенго'
group by 1
--order by ps.total desc

--limit 5


""")

,user_id,original_score,attemps
0,249792088,142,120
1,402555773,34,18
2,299230584,145,50
